<a href="https://colab.research.google.com/github/victoriaazabel/SEdigitalorganizations/blob/main/VictoriaZabel_Assignment5b_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test: Homework 5 but all in one

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
from google.colab import data_table
from sklearn.model_selection import train_test_split
import glob
import os
data_table.enable_dataframe_formatter()


drive.mount('/content/drive')
directory = '/content/drive/My Drive/'
  
# merging the files
joined_files = os.path.join('/content/drive/MyDrive/', "*.csv")
  
# A list of all joined files is returned
joined_list = glob.glob(joined_files)
  
# Finally, the files are joined
all_data_1 = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)

#only gives me medi??




In [ ]:
#all_data_1

In [ ]:
# try again
import os
import pandas as pd

path = "/content/drive/MyDrive/"
files = [file for file in os.listdir(path) if file.endswith(".csv")]

all_table = pd.DataFrame(columns =['Title','Abstract','Index Keywords','Research Field'])

for file in files:
    #try to assign research field name
    research_field = file.rsplit("_", maxsplit=1)[0]
    all_data = pd.read_csv(os.path.join(path, file))
    all_data = all_data.filter(['Index Keywords', 'Title', 'Abstract'])
    all_data['Research Field'] = research_field
    all_table = pd.concat([all_table, all_data])

all_table.head(10)

In [ ]:
#load the dataset once again so you can work with it in the correct format
# string to int so that the research fields 1-26 can be used as labels from datasets 
#import load_dataset
import pandas as pd
from sklearn.preprocessing import LabelEncoder

#dataset5 = load_dataset("all_table")

# create dictionaries for training and conversion
id_label = {0: "AGRI", 1: "ARTS", 2: "BIOC", 3: "BUSI", 4: "CENG", 5: "CHEM", 6: "COMP", 7: "DECI", 8: "DENT", 9: "EART", 10: "ECON", 11: "ENER", 12: "ENGI", 13: "ENVI", 14: "HEAL", 15: "IMMU", 16: "MATE", 17: "MATH", 18: "MEDI", 19: "NEUR", 20: "NURS", 21: "PHAR", 22: "PHYS", 23: "PSYC", 24: "SOCI", 25: "VETE"}
label_id = {"AGRI": 0, "ARTS": 1, "BIOC": 2, "BUSI": 3, "CENG": 4, "CHEM": 5, "COMP": 6, "DECI": 7, "DENT": 8, "EART": 9, "ECON": 10, "ENER": 11, "ENGI": 12, "ENVI": 13, "HEAL": 14, "IMMU": 15, "MATE":16, "MATH": 17, "MEDI": 18, "NEUR": 19, "NURS": 20, "PHAR": 21, "PHYS": 22, "PSYC": 23, "SOCI": 24, "VETE": 25}

# convert str to int again
all_table['Research Field'] = all_table['Research Field'].replace(label_id)

all_table.head(10)

In [ ]:
#split test and train datasets, stratify search?

#train_set, test_set = train_test_split(all_table, test_size = 0.05,  random_state = 42)

In [ ]:
train_set, test_set = train_test_split(all_table,test_size = 0.05, stratify = all_table['Research Field'], random_state=42)

test_set.head(10)


In [ ]:
# Training using the link to hugging face / text classification

!pip install transformers datasets evaluate

from huggingface_hub import notebook_login

notebook_login()

In [ ]:
all_table['Research Field'][4]

In [ ]:
train_set, test_set = train_test_split(all_table,test_size = 0.05, stratify = all_table['Research Field'], random_state=42)

train_set['label'] = train_set['Research Field']

In [ ]:
train_set['label'].head(10)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples['Abstract'], truncation = True)

In [ ]:
# 0.015 instead of the 0.05 that we should've had because it just doesn't work, takes days and crashes
train_mini = train_set.sample(frac = 0.015)
test_mini = test_set.sample(frac = 0.015)

In [ ]:
# create datasets of dataframes for .map 

from datasets import Dataset
train_min = Dataset.from_pandas(train_mini)
test_min = Dataset.from_pandas(test_mini)

In [ ]:
tokenized_trainset = train_min.map(preprocess_function, batched = True)
tokenized_testset = test_min.map(preprocess_function, batched = True)

Map:   0%|          | 0/2203 [00:00<?, ? examples/s]

Map:   0%|          | 0/116 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

In [ ]:
# Now we're getting to the actual training and accuracy

import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis = 1)
    return accuracy.compute(predictions = predictions, references = labels)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels = 26, id2label = id_label, label2id = label_id
)


In [ ]:
#!pip install accelerate -U

In [ ]:
#!pip uninstall -y transformers accelerate
#!pip install accelerate -U

In [ ]:
# the actual model

training_args = TrainingArguments(
    output_dir = "Research-Field-Classification-Model",
    learning_rate = 2e-5,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    num_train_epochs = 2,
    weight_decay = 0.01,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    load_best_model_at_end = True,
    push_to_hub = True,
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_trainset,
    eval_dataset = tokenized_testset,
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics,
)

#trainer.train()

In [ ]:
trainer.train()